## Importing used libraries

In [1]:
# These two lines are necessary only if GemPy is not installed
import sys, os
import numpy as np
sys.path = list(np.insert(sys.path, 0, "../.."))

# Importing GemPy
import gempy as gp

# Importing aux libraries
from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Embedding matplotlib figures in the notebooks
%matplotlib qt5

In [2]:
from gempy.plot import visualization_2d_pro as vv
from gempy.plot import vista

### Initializing the model:

The first step to create a GemPy model is create a gempy.Model object that will contain all the other data structures and necessary functionality.

In addition for this example we will define a regular grid since the beginning. This is the grid where we will interpolate the 3D geological model. GemPy comes with an array of different grids for different pourposes as we will see below. For visualization usually a regular grid is the one that makes more sense.

In [3]:
geo_model = gp.create_model('Model1')
geo_model = gp.init_data(geo_model, extent= [0, 791, 0, 200, -582, 0], resolution=[100, 10, 100])

Active grids: ['regular']


GemPy core code is written in Python. However for efficiency (and other reasons) most of heavy computations happend in optimize compile code, either C or CUDA for GPU. To do so, GemPy rely on the library theano. To guarantee maximum optimization theano requires to compile the code for every Python kernel. The compilation is done by calling the following line at any point (before computing the model):

In [4]:
gp.set_interpolator(geo_model, theano_optimizer='fast_compile', verbose=[])

Setting kriging parameters to their default values.
Compiling theano function...
Level of Optimization:  fast_compile
Device:  cpu
Precision:  float64
Number of faults:  0
Compilation Done!
Kriging values: 
                   values
range             1002.2
$C_o$            23914.4
drift equations      [3]


We can do the same in 3D through pyvista and vtk rendering. Click the qt5 button Back (+Y) to have the same view as in the 2D viwer:

In [5]:
p3d = vista.Vista(geo_model, plotter_type='background', notebook=False, real_time=False)

## Building the model

Now that we have everything initialize we can start the construction of the geological model. 

### Surfaces

GemPy is a surface based interpolator. This means that all the input data we add has to be refered to a surface. The surfaces always mark the bottom of a unit. By default GemPy surfaces are empty:

In [6]:
geo_model.surfaces

,surface,series,order_surfaces,isActive,hasData,color,id


If we do not care about the names and we just want to interpolate a surface we can use:

In [7]:
# Default surfaces:
geo_model.set_default_surfaces()

,surface,series,order_surfaces,isActive,hasData,color,id
0,surface1,Default series,1,nan,nan,#015482,1
1,surface2,Default series,2,nan,nan,#9f0052,2


Now we can start adding data. GemPy input data consist on surface points and orientations (perpendicular to the layers). The 2D plot gives you the X and Z coordinates when hovering the mouse over. We can add a surface point as follows:

In [8]:
# Add a point
geo_model.add_surface_points(X=223, Y=0.01, Z=-94, surface='surface1')


# Plot in 3D
p3d.plot_surface_points()

surface
surface1    #015482
surface2    #9f0052
Name: color, dtype: object


[PolyData (0x1c1f4fb86a8)
   N Cells:	1
   N Points:	1
   X Bounds:	2.230e+02, 2.230e+02
   Y Bounds:	1.000e-02, 1.000e-02
   Z Bounds:	-9.400e+01, -9.400e+01
   N Arrays:	0]

Now we can add the other two points of the layer:

In [14]:
# Add points
geo_model.add_surface_points(X=458, Y=0, Z=-107, surface='surface1')
geo_model.add_surface_points(X=612, Y=0, Z=-14, surface='surface1')

# Plotting

p3d.plot_surface_points()

surface
surface1    #015482
surface2    #9f0052
Name: color, dtype: object


TypeError: 'bool' object is not iterable

In [16]:
%debug

> h:\ssd\pycharmprojects\gempy\gempy\plot\vista.py(646)_plot_surface_points_all()
    644         for fmt in self.model.surfaces.df.groupby('hasData').get_group(True)['surface']:
    645             new_meshes = self.plot_surface_points(fmt, colors=colors, **kwargs)
--> 646             for mesh in new_meshes:
    647                 if mesh is not None:
    648                     meshes.append(mesh)

ipdb> colors
surface
surface1    #015482
surface2    #9f0052
Name: color, dtype: object
ipdb> new_meshes
False
ipdb> exit()


In [18]:
a = p3d._actors[0]

In [23]:
import pandas, numpy
df = pandas.DataFrame(numpy.zeros(shape=[10000, 10]))
dictionary = df.to_dict()

In [29]:
%%timeit
df.loc[1000, 5]

6.8 µs ± 33.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [30]:
%%timeit
dictionary[0][700]

66.8 ns ± 0.245 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


The minimum amount of data to interpolate anything in gempy is:
a) 2 surface points per surface
b) One orientation per series.

Lets add an orientation anywhere in space:

In [12]:
# Adding orientation
geo_model.add_orientations(X=350, Y=0, Z=-300, surface='surface1', pole_vector= (0,0,1))
p3d.plot_data()

AttributeError: 'Vista' object has no attribute 'plot_data'

Now we have enough data for finally interpolate!

In [14]:
gp.compute_model(geo_model)

../../gempy/core/solution.py:266: UserWarning: Attribute error. Using non masked marching cubesmarching_cubes_lewiner() got an unexpected keyword argument 'mask'.
  warnings.warn('Attribute error. Using non masked marching cubes' + str(e)+'.')



Lithology ids 
  [2. 2. 2. ... 2. 2. 2.] 

In [16]:
geo_model.interpolator.theano_graph.a_T.get_value()

array(1.28817491)

In [17]:
geo_model.additional_data.kriging_data

,values
range,1002.2
$C_o$,23914.4
drift equations,[0]
nugget grad,0.01
nugget scalar,1e-06


In [18]:
geo_model.additional_data.rescaling_data

,values
rescaling factor,778
centers,"[417.5, 0.005, -157.0]"


That is, we have interpolated the 3D surface. We can visualize with:

In [19]:
# In 2D
p2d.plot_contacts(ax, cell_number=5)

# In 3D
p3d.plot_surfaces()

,val
0,PolyData (0x7fcfb9cb4648)\n N Cells:\t3044\n ...


## Adding more layers:

So far we only need 2 units defined. The cross-section image that we load have 4 however. Lets add two layers more:

In [20]:
geo_model.surfaces

,surface,series,order_surfaces,isActive,color,id
0,surface1,Default series,1,True,#015482,1
1,surface2,Default series,2,True,#9f0052,2


In [21]:
geo_model.series

,order_series,BottomRelation,isActive
Default series,1,Erosion,True


In [22]:
geo_model.add_surfaces(['surface3', 'basement'])

,surface,series,order_surfaces,isActive,color,id
0,surface1,Default series,1,True,#015482,1
1,surface2,Default series,2,False,#9f0052,2
2,surface3,Default series,3,False,#ffbe00,3
3,basement,Default series,4,True,#728f02,4


### Layer 2

Add the layer next layers:

In [23]:
# Your code here:
geo_model.add_surface_points(X=225, Y=0, Z=-269, surface='surface2')
geo_model.add_surface_points(X=459, Y=0, Z=-279, surface='surface2')


#--------------------
# Plot data
p2d.remove(ax)
p2d.plot_data(ax, cell_number=5, legend='force')
p3d.plot_data()

In [24]:
# Compute model
gp.compute_model(geo_model)

../../gempy/core/solution.py:266: UserWarning: Attribute error. Using non masked marching cubesmarching_cubes_lewiner() got an unexpected keyword argument 'mask'.
  warnings.warn('Attribute error. Using non masked marching cubes' + str(e)+'.')



Lithology ids 
  [4. 4. 4. ... 2. 2. 2.] 

In [28]:
geo_model.interpolator.theano_graph.c_o_T.get_value()

array(30.73830946)

In [26]:
geo_model.additional_data.kriging_data

,values
range,1002.2
$C_o$,23914.4
drift equations,[0]
nugget grad,0.01
nugget scalar,1e-06


In [29]:
# Plot 2D
p2d.plot_lith(ax, cell_number=5)
p2d.plot_contacts(ax, cell_number=5)

# Plot 3D
p3d.plot_surfaces()
p3d.plot_structured_grid(opacity=.2, annotations = {1: 'surface1', 2:'surface2', 3:'surface3'})


(vtkRenderingOpenGL2Python.vtkOpenGLActor)0x7fcfb97d2a08

### Layer 3

In [30]:
# Your code here:
geo_model.add_surface_points(X=225, Y=0, Z=-439, surface='surface3')
geo_model.add_surface_points(X=464, Y=0, Z=-456, surface='surface3')
geo_model.add_surface_points(X=619, Y=0, Z=-433, surface='surface3')

# ------------------
# Plotting data
p2d.remove(ax)
p2d.plot_data(ax, cell_number=5)
p3d.plot_data()

In [31]:
# Computing and plotting 3D
gp.compute_model(geo_model)
p2d.plot_lith(ax, cell_number=5)
p2d.plot_contacts(ax, cell_number=5)

p3d.plot_surfaces()
p3d.plot_structured_grid(opacity=.2, annotations = {1: 'surface1', 2:'surface2', 3:'surface3', 4:'basement'})


../../gempy/core/solution.py:266: UserWarning: Attribute error. Using non masked marching cubesmarching_cubes_lewiner() got an unexpected keyword argument 'mask'.
  warnings.warn('Attribute error. Using non masked marching cubes' + str(e)+'.')


(vtkRenderingOpenGL2Python.vtkOpenGLActor)0x7fcfb97d28e8

In [32]:
geo_model.additional_data

values
Structure isLith                                         True
          isFault                                       False
          number faults                                     0
          number surfaces                                   3
          number series                                     1
          number surfaces per series                      [3]
          len surfaces surface_points               [3, 2, 3]
          len series surface_points                       [8]
          len series orientations                         [1]
Options   dtype                                       float64
          output                                      geology
          theano_optimizer                       fast_compile
          device                                          cpu
          verbosity                                        []
Kriging   range                                        1002.2
          $C_o$                                       23914.4
          drift equations                                 [0]
          nugget grad                                    0.01
          nugget scalar                                 1e-06
Rescaling rescaling factor                                884
          centers                      [421.0, 0.005, -235.0]

In [34]:
geo_model.interpolator.len_series_u

array([0], dtype=int32)

## Faults:

So far the model is simply a depositional unit. GemPy allows for unconformities and faults to build complex models. This input is given by categorical data. In general:

input data (surface points/ orientations) <belong to< surface <belong to< series

And series can be a fault---i.e. offset the rest of surface--- or not. We are going to show how to add a fault as an example.

First we need to add a series:

In [24]:
geo_model.add_series('Fault1')

,order_series,BottomRelation,isActive
Default series,1,Erosion,True
Fault1,2,Erosion,NaN


In [25]:
geo_model.reorder_series(['Fault1', 'Default series'])

,order_series,BottomRelation,isActive
Fault1,1,Erosion,False
Default series,2,Erosion,True


Then define that is a fault:

In [26]:
geo_model.set_is_fault('Fault1')

Fault colors changed. If you do not like this behavior, set change_color to False.


,isFault,isFinite
Fault1,True,False
Default series,False,False


But we also need to add a new surface:

In [27]:
geo_model.add_surfaces('fault1')

,surface,series,order_surfaces,isActive,color,id
0,surface1,Default series,1,True,#015482,1
1,surface2,Default series,2,True,#9f0052,2
2,surface3,Default series,3,True,#ffbe00,3
3,basement,Default series,4,False,#728f02,4
4,fault1,Default series,5,True,#443988,5


And finally assign the new surface to the new series/fault

In [28]:
gp.map_series_to_surfaces(geo_model, {'Fault1':'fault1'})

,surface,series,order_surfaces,isActive,color,id
4,fault1,Fault1,1,True,#443988,1
0,surface1,Default series,1,True,#015482,2
1,surface2,Default series,2,True,#9f0052,3
2,surface3,Default series,3,True,#ffbe00,4
3,basement,Default series,4,False,#728f02,5


Now we can just add input data as before (remember the minimum amount of input data to compute a model):

In [29]:
# Add input data of the fault
geo_model.add_surface_points(X=550, Y=0, Z=-30, surface='fault1')
geo_model.add_surface_points(X=650, Y=0, Z=-200, surface='fault1')
geo_model.add_orientations(X=600, Y=0, Z= -100, surface='fault1', pole_vector=(.3,0,.3))

# Plotting Inpute data
p2d.remove(ax)
p2d.plot_data(ax, cell_number=5)
p3d.plot_data()

And now is computing as before:

In [30]:
# Compute
gp.compute_model(geo_model)

# Plot
p2d.plot_lith(ax, cell_number=5)
p2d.plot_contacts(ax, cell_number=5)
p3d.plot_surfaces()
p3d.plot_structured_grid(opacity=.2, annotations = {2: 'surface1', 3:'surface2', 4:'surface3', 5:'basement'})

(vtkRenderingOpenGL2Python.vtkOpenGLActor)0x7fe4721852e8

As you can see now instead of having folding layers we have a sharp jump. Building on this you can pretty much any model you can imagine.

# Additional features:

Over the years we have built a bunch of assets integrate with gempy. Here we will show some of them:

### Topogragpy

GemPy has a built-in capanilities to read and manipulate topographic data (through gdal). To show an example we can just create a random topography:

In [31]:
## Adding random topography
geo_model.set_topography(source='random',fd=1.9, d_z=np.array([-150, 0]), resolution=np.array([200,200]))

Active grids: ['regular' 'topography']


Grid Object. Values: 
array([[   3.955     ,   10.        , -579.09      ],
       [   3.955     ,   10.        , -573.27      ],
       [   3.955     ,   10.        , -567.45      ],
       ...,
       [ 779.17474874,  190.        ,  -61.16873539],
       [ 783.10987437,  190.        ,  -60.02975807],
       [ 787.045     ,  190.        ,  -59.76255033]])

The topography can we visualize in both renderers:

In [32]:
p2d.plot_topography(ax, cell_number=5)
p3d.plot_topography(scalars='topography')

p1 [0.0, 100.0] p2 [791.0, 100.0]


(vtkRenderingOpenGL2Python.vtkOpenGLActor)0x7fe47216aca8

But also allows us to compute the geological map of an area:

In [33]:
gp.compute_model(geo_model)
p3d.plot_topography(scalars='geo_map')

(vtkRenderingOpenGL2Python.vtkOpenGLActor)0x7fe472185a68

### Gravity inversion

GemPy also allows for inversions (in production only gravity so far). We can see a small demo how this works.

The first thing to do is to assign densities to each of the units:

In [34]:
geo_model.surfaces.add_surfaces_values([0, 2.6, 2.4, 3.2, 3.6], ['density'])

,surface,series,order_surfaces,isActive,color,id,density
4,fault1,Fault1,1,True,#443988,1,0
0,surface1,Default series,1,True,#015482,2,2.6
1,surface2,Default series,2,True,#9f0052,3,2.4
2,surface3,Default series,3,True,#ffbe00,4,3.2
3,basement,Default series,4,True,#728f02,5,3.6


Also we can create a centered grid around a device for precision:

In [35]:
geo_model.set_centered_grid(centers= [[400, 0, 0]],  resolution = [10, 10, 100], radio=800)

Active grids: ['regular' 'topography' 'centered']


Grid Object. Values: 
array([[    3.955     ,    10.        ,  -579.09      ],
       [    3.955     ,    10.        ,  -573.27      ],
       [    3.955     ,    10.        ,  -567.45      ],
       ...,
       [ 1200.        ,   800.        ,  -922.71624587],
       [ 1200.        ,   800.        ,  -964.3665184 ],
       [ 1200.        ,   800.        , -1008.        ]])

We need to modify the compile code:

In [36]:
gp.set_interpolator(geo_model, output=['gravity'], theano_optimizer='fast_run', )

Compiling theano function...
Level of Optimization:  fast_run
Device:  cpu
Precision:  float64
Number of faults:  1
Compilation Done!


But now additionally to the interpolation we also compute the forward gravity of the model (at the point XYZ = 400, 0, 0)

In [37]:
gp.compute_model(geo_model)
geo_model.solutions.fw_gravity

../../gempy/core/solution.py:266: UserWarning: Attribute error. Using non masked marching cubesmarching_cubes_lewiner() got an unexpected keyword argument 'mask'.
  warnings.warn('Attribute error. Using non masked marching cubes' + str(e)+'.')


array([-803.64195548])

We can visualize it it better in the following figure. The aim of an inversion is to find the set of parameters that fit a measured point the better. In this example the red x symbolize the measured gravity while the blue dots are the current gravity (and previous) fw gravity values. The widget moves up and down the surface 3

In [38]:
# Creating new figure
p_grav = vv.Plot2D(geo_model)
p_grav.create_figure((6, 9))

# Adding section for model
ax2 = p_grav.add_section(cell_number=1, direction='y', ax_pos=211)

# Adding section for gravity 
ax3 = p_grav.add_section(ax_pos=414)

# Plotting model in section
p2d.plot_data(ax2, cell_number=5, legend='force')
p2d.plot_lith(ax2, cell_number=5)
p2d.plot_contacts(ax2, cell_number=5)
ax2.plot(400, 0, '^', markersize=40, c='red')

# Plotting initial values of the gravity axes
target_grav = -810
ax3.tick_params( bottom=False)
ax3.spines['top'].set_visible(False)
ax3.spines['bottom'].set_visible(False)
ax3.spines['right'].set_visible(False)
ax3.plot(0, target_grav, 'X', label='Target Gravity', markersize=4, c='red')
ax3.plot(1, geo_model.solutions.fw_gravity, 'o', label='Current Gravity', markersize=4, c='blue')
ax3.set_ylabel('grav')

# We store the original values of z for the surface 3
Z_ori = geo_model.surface_points.df.loc[[5,6,7], 'Z'].copy()
# init a list to store grav
grav_ = []

# Function that modify the model, compute it and plot
def gravity_invert(dz):
    new_z = Z_ori + dz
    geo_model.modify_surface_points(indices=[5, 6, 7], Z= new_z)
    gp.compute_model(geo_model)
    grav_.append(geo_model.solutions.fw_gravity[0])
    
    p2d.remove(ax2)
    p2d.plot_data(ax2, cell_number=5)
    p2d.plot_lith(ax2, cell_number=5)
    p2d.plot_contacts(ax2, cell_number=5)
    ax3.plot(np.arange(len(grav_))+1, grav_, 'o', label='Current Gravity', markersize=4, c='blue')
    ax3.set_xlim(-1, len(grav_)+1)
    
interact(gravity_invert, dz=(-200, 200, 10))

0.7357774968394437


interactive(children=(IntSlider(value=0, description='dz', max=200, min=-200, step=10), Output()), _dom_classe…

<function __main__.gravity_invert(dz)>

## More advanced model:

In [39]:
geo_model = gp.create_model('Moureze')
geo_model = gp.init_data(geo_model, extent= [0, 3630, -0.5, 0.5, -1084, 0], resolution=[321, 21, 91])

Active grids: ['regular']


In [40]:
img = mpimg.imread('model2.png')

In [41]:
p2d = vv.Plot2D(geo_model)
p2d.create_figure((13, 6))
ax4 = p2d.add_section(cell_number=11)

0.2986225895316804


In [42]:
ax4.imshow(img, origin='upper', alpha=.8, extent = (0, 3630, -1084,0))